In [734]:
import pandas as pd
import numpy as np

# data = pd.read_excel('../data/master.xlsx')
# data = pd.read_excel('../data/master.xlsx').sample(1000)
data = {'A': [3, 3, 2], 'B': [3, 3, 4], 'C': [7, 7, 7]}
data = pd.DataFrame(data)

minSup = 1
numDims = len(data.columns)
outputRec = pd.DataFrame({
    "Combination": [],
    "Count": []
})


In [735]:
data.columns

Index(['A', 'B', 'C'], dtype='object')

In [736]:
data.head()

,A,B,C
0,3,3,7
1,3,3,7
2,2,4,7


In [737]:
# unique_counts = data.nunique()
# col_dict = dict(zip(unique_counts.index, unique_counts.values))
# sorted_cols = sorted(col_dict, key=col_dict.get, reverse=True)
# data = data[sorted_cols]

In [738]:
data.head()

,A,B,C
0,3,3,7
1,3,3,7
2,2,4,7


In [739]:
# def Partition(input_df, dim, start_row, end_row, dataCount):
#     subset_df = input_df.iloc[start_row:end_row + 1, :]
#     sorted_df = subset_df.sort_values(by=input_df.columns[dim])
#     input_df.iloc[start_row:end_row + 1, :] = sorted_df.values
#     counts = sorted_df.iloc[:, dim].value_counts().sort_index().tolist()
#     dataCount.extend(counts)


In [764]:
import heapq
import os
import openpyxl

def Partition(file_name, dim, start_row, end_row, dataCount):
    workbook = openpyxl.load_workbook(file_name)
    worksheet = workbook.active
    tempFileNames = []

    line_number = 0
    pageSize = 2
    pageIndex = 0
    df = pd.DataFrame()
    for row in worksheet.iter_rows(min_row=start_row+2, max_row=end_row+2, values_only=True):
        if line_number >= pageSize:
            df.sort_values(by=df.columns[dim], inplace=True)
            tempFileName = "../data/temporary/" + str(pageIndex) + ".xlsx"
            df.to_excel(tempFileName, index=False, header=False)
            tempFileNames.append(tempFileName)
            line_number = 0
            pageIndex += 1
            df = pd.DataFrame()

        df = df.append(pd.DataFrame([row]), ignore_index=True)
        line_number += 1

    if line_number > 0:
        df.sort_values(by=df.columns[dim], inplace=True)
        tempFileName = "../data/temporary/" + str(pageIndex) + ".xlsx"
        df.to_excel(tempFileName, index=False, header=False)
        tempFileNames.append(tempFileName)
        pageIndex += 1

    print(tempFileNames)

    print("the above code is optimised as our dataset did not fit the main memory")


    min_heap = []

    file_handles = [openpyxl.load_workbook(tempFileName).active for tempFileName in tempFileNames]
    iterators = [file.iter_rows(values_only=True) for file in file_handles]
    current_row = start_row+2

    # Push the <value, pageid, row> of the first row of each file_handle to the min heap
    for i, iterator in enumerate(iterators):
        first_row = next(iterator)
        heapq.heappush(min_heap, (first_row[dim], i, first_row))


    # # While the heap is not empty
    # while len(min_heap) > 0:
    #     value, file_index, row = heapq.heappop(min_heap)

    #     # Rewrite the current row of the original file
    #     original_row = worksheet[row]
    #     original_row[dim].value = value

    #     dataCountLength = len(dataCount)
    #     if dataCountLength == 0:
    #         dataCount[value] = 1
    #     elif value in dataCount:
    #         dataCount[value] += 1
    #     else:
    #         dataCount[value] = 1

    #     # Read the next row from the file of index page_index if it is not empty
    #     try:
    #         next_row = next(file_handles[file_index].iter_rows(values_only=True))
    #         heapq.heappush(min_heap, (next_row[dim], file_index, current_row))
    #         current_row += 1
    #     except StopIteration:
    #         # The file is empty, no more rows to read from it
    #         continue


    # # Delete all the temporary files
    # for tempFileName in tempFileNames:
    #     os.remove(tempFileName)

    # # Save the changes to the original workbook
    # workbook.save(file_name)

    # #push the <value, pageid, row> of the first row of each file_handle to the min heap


    # # while the heap is not empty
    # while len(min_heap) > 0:
    #     value, file_index, row = heapq.heappop(min_heap)
        
    #     #rewrite the current row of original file

    #     dataCountLength = len(dataCount)
    #     if dataCountLength == 0:
    #         dataCount.append({
    #             "value": value,
    #             "count": 1
    #         })
    #     elif dataCount[dataCountLength - 1]["value"] == value:
    #         dataCount[dataCountLength - 1]["count"] += 1
    #     else:
    #         dataCount.append({
    #             "value": value,
    #             "count": 1
    #         })

    #     # read the next row from the file of index page_index if it is not empty
        


    # # close all the files

    # # delete all the temporary files
    # for tempFileName in tempFileNames:
    #     os.remove(tempFileName)

['../data/temporary/0.xlsx', '../data/temporary/1.xlsx']
the above code is optimised as our dataset did not fit the main memory
(9, 2, 2)
(11, 3, 7)


/tmp/ipykernel_8664/675156592.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([row]), ignore_index=True)
/tmp/ipykernel_8664/675156592.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([row]), ignore_index=True)


In [757]:
Partition("../data/inputs/test.xlsx", 0, 0, 3, {})

['../data/temporary/0.xlsx', '../data/temporary/1.xlsx']
the above code is optimised as our dataset did not fit the main memory
('A', 'D', 'C')
('A', 'D', 'C')


/tmp/ipykernel_8664/2255354592.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([row]), ignore_index=True)
/tmp/ipykernel_8664/2255354592.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([row]), ignore_index=True)


In [742]:
# def BottomUpCube(input, dim, start_row, end_row, currentCombination, numDims, minSup):
#     global outputRec
#     for d in range(dim, numDims):
#         dataCount = {}
#         Partition(input, d, start_row, end_row, dataCount)

#         k = start_row
#         for value, c in dataCount.items():
#             if c >= minSup:
#                 outputRec = pd.concat([pd.DataFrame(outputRec), pd.DataFrame([{
#                     "Combination": currentCombination + [value], 
#                     "Count": c
#                     }])], ignore_index=True)

#                 BottomUpCube(input, d + 1, k, k + c - 1, currentCombination + [value], numDims, minSup)

#             k += c    

In [743]:
# count = len(data)
# if count >= minSup:
#     outputRec = pd.concat([pd.DataFrame(outputRec), pd.DataFrame([{"Combination": [], "Count": count}])], ignore_index=True)

In [744]:
# fileName = "../data/inputs/test.xlsx"

In [745]:
# BottomUpCube(fileName, 0, 0, 3, [], numDims, minSup)

In [746]:
# print(outputRec)

In [747]:
# data.head()